# Example Functional connectivity in Python 
####  this code: 
####  1) Imports the preprocessed and epoched python EEG data
####  2) Calculates the wPLI 

# ADD INFORMATION HERE




In [1]:
import matplotlib
import mne

# Ensure Matplotlib uses the Qt5Agg backend, 
# which is the best choice for MNE-Python's 
# interactive plotting functions.
matplotlib.use('Qt5Agg')

### Which subject do you want to preprocess? 

In [2]:
ID = "test"
task = "Base"

In [3]:
path_epochs = "testdata/sub-{}/eeg/epochs_{}_{}.fif".format(ID,ID,task)
path_epochs

'testdata/sub-test/eeg/epochs_test_Base.fif'

## Load the preprocessed and epoched data

In [4]:
epochs = mne.read_epochs(path_epochs)
epochs = epochs.drop_channels(epochs.info['bads'])

Reading testdata/sub-test/eeg/epochs_test_Base.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.00 ms
        0 CTF compensation matrices available


<ipython-input-4-04d9a8f44741>:1: RuntimeWarning: This filename (testdata/sub-test/eeg/epochs_test_Base.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(path_epochs)


Not setting metadata
Not setting metadata
29 matching events found
No baseline correction applied
0 projection items activated


#### The data is cropped in 10 s windows. There are 29 windos. 
### Here, you get more information about the data

In [6]:
sfreq = epochs.info['sfreq']
sfreq

250.0

## calculate the wPLI 

In [7]:
# define parameters: 
l_freq = 7
h_freq = 13
window_length = 10
step_size = 10


In [8]:
# filter the epochs
epochs_filterted = epochs.filter(l_freq, h_freq)

Setting up band-pass filter from 7 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 413 samples (1.652 sec)



In [9]:
# this funcrion calculates the wpLI in a time-resolved way. 
wPLI = get_wPLI(epochs, sfreq, window_length, )


NameError: name 'get_wPLI' is not defined

In [ ]:
# visualize the data here

In [ ]:
#   .... 